In [1]:
def monthToNum(shortMonth):
    return {
            'Jan': 1,
            'Fev': 2,
            'Mar': 3,
            'Abr': 4,
            'Mai': 5,
            'Jun': 6,
            'Jul': 7,
            'Ago': 8,
            'Set': 9, 
            'Out': 10,
            'Nov': 11,
            'Dez': 12
    }[shortMonth]

In [2]:
def function(df_in):
    
    # criando uma cópia do dataframe original para não manipular o original
    df = df_in.copy()
    
    # ajuste das colunas para o dataframe final
    column_names_to_change = {'COMBUSTÍVEL': 'product', 'ESTADO': 'uf', 'UNIDADE': 'unit' }
    final = pd.DataFrame(columns = ['product', 'ANO', 'uf', 'unit', 'month', 'volume'])

    # mudando nome da colunas e retirando colunas desnecessárias
    df.rename(columns = column_names_to_change, inplace = True)
    df.drop('REGIÃO', inplace = True, axis=1)
    
    # ajustar o nome de cada combustivel
    df['product'] = df['product'].apply(lambda x: x.split(' (')[0])

    # nome das colunas separadas em duas partes: x = left & y = right
    x = df.columns[0:4] # combustivel, ano, estado e unidade
    y = df.columns[4:-1] # 12 meses do ano
    
    valores_errados = 0
    
    # iterando no dataframe inteiro
    for i in range(len(df)):
        # coletando o lado esquerdo de cada linha e copiando a mesma para os 12 meses do ano
        aux_left = pd.DataFrame(df[x].loc[i]).transpose()
        aux_left = pd.concat([aux_left]*len(y), ignore_index=True)
        
        # coletando o lado direito de cada linha
        aux_right = pd.DataFrame(df[y].loc[i]).reset_index(drop = False)
        aux_right.columns = ['month', 'volume']
        
        # checando se o valor coletado dos meses é igual ao total informado originalmente
        if sum(aux_right['volume']) != df['TOTAL'].loc[i]:
            valores_errados += 1
        
        # juntando o lado direito e esquerdo novamente
        result = pd.concat([aux_left, aux_right], axis=1)
        
        # "appendando" a linha formatada para o dataframe final
        final = final.append(result, ignore_index = True)
    
    
    # convertendo o nome do mês para o número
    final['month'] = final['month'].apply(lambda x: monthToNum(x))
    aux = []
    for i in range(len(final)):
        aux.append(str(final['ANO'][i]) + '-' + str(final['month'][i]))
        
    # criação da coluna year_month e excluindo as colunas desnecessárias
    final['year_month'] = aux
    final.drop(['ANO', 'month'], inplace = True, axis=1)
    
    # criação da coluna created_at
    created_at = [pd.Timestamp(time.time(), unit='s')]*len(final)
    final['created_at'] = created_at
    
    print("Quantidade de Totais errados:", valores_errados)
    
    return final[['year_month', 'uf', 'product', 'unit', 'volume', 'created_at']]

In [3]:
import numpy as np

import pandas as pd

import time

In [4]:
oil_df = pd.read_excel('vendas-combustiveis-m3.xls', sheet_name = 'oil_uf_product')
diesel_df = pd.read_excel('vendas-combustiveis-m3.xls', sheet_name = 'diesel_uf_type')

oil_df.head()

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,UNIDADE,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,GASOLINA C (m3),2000,REGIÃO NORTE,RONDÔNIA,m3,9563.263,11341.229,9369.746,10719.983,11165.968,12312.451,11220.970,12482.281,13591.122,11940.570,11547.576,10818.094,136073.253
1,GASOLINA C (m3),2000,REGIÃO NORTE,ACRE,m3,3065.758,3495.290,2946.930,3023.920,3206.930,3612.580,3264.460,3835.740,3676.571,3225.610,3289.718,3358.346,40001.853
2,GASOLINA C (m3),2000,REGIÃO NORTE,AMAZONAS,m3,17615.604,20258.200,18741.344,19604.023,20221.674,20792.616,19912.898,21869.338,21145.643,20633.175,20766.918,21180.919,242742.352
3,GASOLINA C (m3),2000,REGIÃO NORTE,RORAIMA,m3,3259.300,3636.216,3631.569,3348.416,3394.016,4078.616,3346.616,4029.900,4358.516,3716.032,3200.400,3339.332,43338.929
4,GASOLINA C (m3),2000,REGIÃO NORTE,PARÁ,m3,28830.479,32297.047,27310.979,29396.384,26511.009,36553.250,31807.840,31009.972,29755.907,28661.951,28145.784,29294.796,359575.398


In [5]:
oil_final = function(oil_df)

oil_final

Quantidade de Totais errados: 216


,year_month,uf,product,unit,volume,created_at
0,2000-1,RONDÔNIA,GASOLINA C,m3,9563.263000,2021-08-02 11:54:31.704335213
1,2000-2,RONDÔNIA,GASOLINA C,m3,11341.229000,2021-08-02 11:54:31.704335213
2,2000-3,RONDÔNIA,GASOLINA C,m3,9369.746000,2021-08-02 11:54:31.704335213
3,2000-4,RONDÔNIA,GASOLINA C,m3,10719.983000,2021-08-02 11:54:31.704335213
4,2000-5,RONDÔNIA,GASOLINA C,m3,11165.968000,2021-08-02 11:54:31.704335213
...,...,...,...,...,...,...
54427,2020-8,DISTRITO FEDERAL,GLP,m3,15358.490942,2021-08-02 11:54:31.704335213
54428,2020-9,DISTRITO FEDERAL,GLP,m3,13937.451087,2021-08-02 11:54:31.704335213
54429,2020-10,DISTRITO FEDERAL,GLP,m3,NaN,2021-08-02 11:54:31.704335213
54430,2020-11,DISTRITO FEDERAL,GLP,m3,NaN,2021-08-02 11:54:31.704335213


In [6]:
oil_final.dtypes

year_month            object
uf                    object
product               object
unit                  object
volume               float64
created_at    datetime64[ns]
dtype: object

In [7]:
diesel_final = function(diesel_df)

diesel_final

Quantidade de Totais errados: 135


,year_month,uf,product,unit,volume,created_at
0,2013-1,RONDÔNIA,ÓLEO DIESEL S-10,m3,3517.60,2021-08-02 11:54:42.672039509
1,2013-2,RONDÔNIA,ÓLEO DIESEL S-10,m3,3681.70,2021-08-02 11:54:42.672039509
2,2013-3,RONDÔNIA,ÓLEO DIESEL S-10,m3,4700.67,2021-08-02 11:54:42.672039509
3,2013-4,RONDÔNIA,ÓLEO DIESEL S-10,m3,5339.20,2021-08-02 11:54:42.672039509
4,2013-5,RONDÔNIA,ÓLEO DIESEL S-10,m3,6166.40,2021-08-02 11:54:42.672039509
...,...,...,...,...,...,...
12955,2020-8,DISTRITO FEDERAL,ÓLEO DIESEL,m3,0.00,2021-08-02 11:54:42.672039509
12956,2020-9,DISTRITO FEDERAL,ÓLEO DIESEL,m3,0.00,2021-08-02 11:54:42.672039509
12957,2020-10,DISTRITO FEDERAL,ÓLEO DIESEL,m3,NaN,2021-08-02 11:54:42.672039509
12958,2020-11,DISTRITO FEDERAL,ÓLEO DIESEL,m3,NaN,2021-08-02 11:54:42.672039509


In [8]:
diesel_final.dtypes

year_month            object
uf                    object
product               object
unit                  object
volume               float64
created_at    datetime64[ns]
dtype: object